In [1]:
import sys
sys.path.append('../../')
import bemppUQ
import numpy as np
import bempp.api 
from bemppUQ.functions import tangential_trace, neumann_trace
from bemppUQ.utils.login import rescale
from bemppUQ.operators.maxwell import multitrace_identity, multitrace_operator, assemble_operators, evaluate_far_field, evaluate_far_field_sd

from bempp.api.assembly.blocked_operator import BlockedOperator
from bemppUQ.utils.mie import bhmie
from bemppUQ.preconditioning.osrc import osrc_MtE
from bemppUQ.utils.login import rescale, gmres
from bempp.api.assembly.blocked_operator import \
        coefficients_of_grid_function_list, \
        projections_of_grid_function_list, \
        grid_function_list_from_coefficients

In [10]:
# FOA PRIMAL

k_int = None
k_ext = None
n = None
mu_ext = None
mu_int = None
eps_ext = None
eps_int = None
mu_rel = None
eps_rel = None

eps_rel = 1.9
mu_rel = 1.

k_ext = 1.047197551196598
k_int = k_ext * np.sqrt(eps_rel * mu_rel)
n = np.sqrt(eps_rel * mu_rel)


print(k_int, k_ext, n, mu_rel, eps_rel)

k_int = None
k_ext = None
n = None
mu_ext = None
mu_int = None
eps_ext = None
eps_int = None
mu_rel = None
eps_rel = None

# TP 
mu_ext = 1
mu_int = 1

k_ext = 1.047197551196598
n = 1.311
k_int = n*k_ext
mu_rel = mu_int / mu_ext
eps_rel = k_int ** 2 / k_ext ** 2 * mu_rel


print(k_int, k_ext, n, mu_rel, eps_rel)


1.4434622098763403 1.047197551196598 1.378404875209022 1.0 1.9
1.37287598961874 1.047197551196598 1.311 1.0 1.7187209999999995


In [11]:
bempp.api.global_parameters.hmat.eps = 1E-5

m_ext = 1
m_int = 1

k_ext = 1.047197551196598
n = 1.311
k_int = n*k_ext

precision = 30
h = 2*np.pi/(precision*k_ext)

maxiter = 10000
restart = 10000
tolerance =  1e-04

mu_rel = m_int / m_ext 


k_int = k_ext * np.sqrt(eps_rel * mu_rel)
eps_rel = k_int ** 2 / k_ext ** 2 * mu_rel




number_of_angles = 3601
angles = np.pi * np.linspace(0, 2, number_of_angles)
unit_points = np.array([np.cos(angles), np.sin(angles), np.zeros(number_of_angles)])

ka = k_ext
refIndex = n
numAngles = 901

[s1,s2,qext,qsca,qback,gsca]=bhmie(ka,refIndex,numAngles)

grid = bempp.api.shapes.sphere(h=1)


def incident_field_z(x):
    return np.array([0. * x[0], 0. * x[0], np.exp(1j * k_ext * x[0])])

# Incident field polarized in y-direction
def incident_field_y(x):
    return np.array([0. * x[0], np.exp(1j * k_ext * x[0]), 0. * x[0]])

# Dirichlet trace
def dirichlet_trace_z(x, n, domain_index, result):
    incident_field_z = np.array([0. * x[0], 0. * x[0], np.exp(1j * k_ext * x[0])])
    result[:] = np.cross(incident_field_z, n)

def dirichlet_trace_y(x, n, domain_index, result):
    incident_field_y = np.array([0. * x[0], np.exp(1j * k_ext * x[0]), 0. * x[0]])
    result[:] = np.cross(incident_field_y, n)

def neumann_trace_z(x, n, domain_index, result):
    curl_z = np.array([x[0] * 0. ,-1j * k_ext * np.exp(1j * k_ext * x[0]), x[0] * 0.])
#     with objmode():
    result[:] = (1/(1j * k_ext)) * np.cross(curl_z, n)

def neumann_trace_y(x, n, domain_index, result):
    curl_y = np.array([x[0] * 0., x[0] * 0. ,1j * k_ext * np.exp(1j * k_ext * x[0])])
#     with objmode():
    result[:] = (1/(1j * k_ext)) * np.cross(curl_y,n)

In [12]:

RWG = bempp.api.function_space(grid, 'RWG', 0)
SNC = bempp.api.function_space(grid, 'SNC', 0)
print(RWG.global_dof_count * 2 , 'N')

op_int = BlockedOperator(2,2)
op_ext = BlockedOperator(2,2)
op_osrc_int = BlockedOperator(2,2)
op_osrc_ext = BlockedOperator(2,2)

efield_e = bempp.api.operators.boundary.maxwell.electric_field(RWG, RWG, SNC, k_ext)
mfield_e = bempp.api.operators.boundary.maxwell.magnetic_field(RWG, RWG, SNC, k_ext)

efield_i = bempp.api.operators.boundary.maxwell.electric_field(RWG, RWG, SNC, k_int)
mfield_i = bempp.api.operators.boundary.maxwell.magnetic_field(RWG, RWG, SNC, k_int)

osrc_ext = osrc_MtE(grid, k_ext)
osrc_int = osrc_MtE(grid, k_int)

op_ext[0,0] = mfield_e
op_ext[0,1] = efield_e
op_ext[1,0] = -1*efield_e
op_ext[1,1] = mfield_e

op_int[0,0] = mfield_i
op_int[0,1] = efield_i
op_int[1,0] = -1*efield_i
op_int[1,1] = mfield_i

op_osrc_ext[0,1] = osrc_ext
op_osrc_ext[1,0] = -osrc_ext

op_osrc_int[0,1] = osrc_int
op_osrc_int[1,0] = -osrc_int

op_osrc_ext = rescale(op_osrc_ext, k_ext, m_ext)
op_osrc_int = rescale(op_osrc_int, k_int, m_int)

op_osrc = op_osrc_ext + op_osrc_int

op_ext = rescale(op_ext, k_ext, m_ext)
op_int = rescale(op_int, k_int, m_int)

op = op_ext + op_int

identity = BlockedOperator(2,2)
identity[0, 0] = bempp.api.operators.boundary.sparse.identity(RWG, RWG, SNC)
identity[1, 1] = bempp.api.operators.boundary.sparse.identity(RWG, RWG, SNC)

####
transmission_operators = assemble_operators(grid, k_int, k_ext, unit_points, osrc=True, spaces='maxwell_primal')
far_field, solution = evaluate_far_field(transmission_operators, eps_rel, mu_rel, k_ext, polarization, direction)


#Discretize the incident field
incident_dirichlet_z = bempp.api.GridFunction(RWG, fun=dirichlet_trace_z)
incident_neumann_z = (k_ext/m_ext) * bempp.api.GridFunction(RWG, fun=neumann_trace_z)

incident_dirichlet_y = bempp.api.GridFunction(RWG, dual_space=RWG, fun=dirichlet_trace_y)
incident_neumann_y = (k_ext/m_ext) * bempp.api.GridFunction(RWG, fun=neumann_trace_y)

op_rhs = 0.5 * identity - op_int

rhs_z = [incident_dirichlet_z, incident_neumann_z]
rhs_z = op_rhs * rhs_z

rhs_y = [incident_dirichlet_y, incident_neumann_y]
rhs_y = op_rhs * rhs_y

b_y = projections_of_grid_function_list(rhs_y, op.dual_to_range_spaces)
b_z = projections_of_grid_function_list(rhs_z, op.dual_to_range_spaces)

op_osrc_wf = op_osrc.weak_form()
op_wf = op.weak_form()

x_y, info, res_y = gmres(op_osrc_wf * op_wf ,op_osrc_wf * b_y, tol=tolerance, return_residuals=True,  maxiter=maxiter, restart=restart)
x_z, info, res_z = gmres(op_osrc_wf * op_wf ,op_osrc_wf * b_z, tol=tolerance, return_residuals=True,  maxiter=maxiter, restart=restart)

sol_y = grid_function_list_from_coefficients(x_y, op.domain_spaces)
sol_z = grid_function_list_from_coefficients(x_z, op.domain_spaces)

144 N


NameError: name 'polarization' is not defined

In [ ]:
far_field_y = np.zeros((3, number_of_angles), dtype='complex128')
far_field_z = np.zeros((3, number_of_angles), dtype='complex128')
# bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'

electric_far = bempp.api.operators.far_field.maxwell.electric_field(RWG, unit_points, k_ext)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(RWG, unit_points, k_ext)    
far_field_y += -electric_far * (m_ext/k_ext *sol_y[1]) - magnetic_far * sol_y[0]
far_field_z += -electric_far * (m_ext/k_ext *sol_z[1]) - magnetic_far * sol_z[0]

#far_field_z

A22 = -1j * k_ext * far_field_z[2,:]   # -1j * kExt is to make agree with eqn 3.21 p70 Bohren and Huffman
A12 = -1j * k_ext * (- np.sin(angles) * far_field_z[0,:] + np.cos(angles) * far_field_z[1,:])

A11 = -1j * k_ext * (- np.sin(angles) * far_field_y[0,:] + np.cos(angles) * far_field_y[1,:])
A21 = -1j * k_ext * far_field_y[2,:]


In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['figure.figsize'] = (15, 4) # Increase the figure size in the notebook
params = {'legend.fontsize': 'x-large', 'axes.labelsize': 'x-large', 'axes.titlesize': 'x-large',
         'xtick.labelsize': 'x-large', 'ytick.labelsize': 'x-large'}
plt.rcParams.update(params)

fig, axes = plt.subplots(1, 2, sharex=True, sharey=False)

f0 = axes[0].semilogy(angles[:1801],abs(s2)**2,'-r',angles[:1801],abs(A11[:1801])**2,'--b')
axes[0].set_title('$A_{11}$: Mie (red) vs. BEM (blue)')

f1 = axes[1].semilogy(angles[:1801],abs(s1)**2,'-r',angles[:1801],abs(A22[:1801])**2,'--b')
axes[1].set_title('$A_{22}$: Mie (red) vs. BEM (blue)')


plt.show()